In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [4]:
spark = SparkSession \
       .builder \
       .appName("parking_vio") \
       .getOrCreate()
spark       

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
file_location = "/content/drive/My Drive/segmentdata.csv"
file_type = "csv"

# CSV options
infer_schema = True
first_row_is_header = True
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [7]:
#df=spark.read.csv("segmentdata.csv",inferSchema=True,header=True)
df.show(5,False)

+-------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+---------------+--------------+---------------------------------+----------------+------------------+
|_c0    |Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Issuer Precinct|Issuer Command|Violation In Front Of Or Opposite|Violation County|Violation Location|
+-------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+---------------+--------------+---------------------------------+----------------+------------------+
|4031366|45                |56        |1853      |36            |1009             |593         |12            |260         |265         |243         |114            |2644          |1                                |5               |113         

In [8]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Registration State: integer (nullable = true)
 |-- Plate Type: integer (nullable = true)
 |-- Issue Date: integer (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: integer (nullable = true)
 |-- Vehicle Make: integer (nullable = true)
 |-- Issuing Agency: integer (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Command: integer (nullable = true)
 |-- Violation In Front Of Or Opposite: integer (nullable = true)
 |-- Violation County: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)



In [9]:
#df.head()

In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
parking_df = df.toPandas()
parking_df.head()

,_c0,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Issuer Precinct,Issuer Command,Violation In Front Of Or Opposite,Violation County,Violation Location
0,4031366,45,56,1853,36,1009,593,12,260,265,243,114,2644,1,5,113
1,697788,45,56,1201,20,1009,1957,12,5695,1305,6246,67,2632,1,2,66
2,4067444,45,18,452,36,1274,1056,12,334,1210,1161,17,2605,1,4,16
3,6267274,9,56,1131,20,1,3766,12,4961,2342,2566,84,2631,3,2,83
4,5740722,45,56,1855,73,1009,1056,12,1543,2989,4435,120,2633,1,6,119


In [12]:
df.describe().toPandas()

,summary,_c0,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Issuer Precinct,Issuer Command,Violation In Front Of Or Opposite,Violation County,Violation Location
0,count,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870,1513870
1,mean,4256927.634950161,42.677176375778636,47.57235165502983,916.2061352692107,35.82126206345327,665.5732519965387,1905.4985560186806,11.741475820248766,2267.027690620727,2005.092284013819,1990.739388454755,55.877301881931736,2410.0439443281125,1.6596517534530706,3.35797789770588,54.661618236704605
2,stddev,2457673.1967418436,8.369252428019957,15.942259567536535,534.5794256872383,19.50424786018186,510.2012225349831,1119.5057976211133,0.8213391001711791,1914.456762421452,1969.8883030609118,1962.4778141784425,47.894031549841216,665.1683471938428,0.8909040294581622,1.4104490532008207,37.94045015984362
3,min,1,0,0,1,0,0,7,0,0,0,0,0,0,1,1,0
4,max,8513868,66,81,1856,98,1364,4504,13,6475,6796,6713,606,2800,5,6,179


In [13]:
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+---+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+---------------+--------------+---------------------------------+----------------+------------------+
|_c0|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Issuer Precinct|Issuer Command|Violation In Front Of Or Opposite|Violation County|Violation Location|
+---+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+---------------+--------------+---------------------------------+----------------+------------------+
|  0|                 0|         0|         0|             0|                0|           0|             0|           0|           0|           0|              0|             0|                                0|               0|                 0|
+---+---

In [14]:
from sklearn.model_selection import train_test_split


In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [16]:
#import pandas as pd
#data = pd.read_csv('segmentdata.csv', index_col=0)

In [17]:
train_string_columns = []

for col, dtype in df.dtypes:
    if dtype == 'string':
        train_string_columns.append(col)

In [18]:
train_string_columns

[]

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+'_index', handleInvalid='keep').fit(df) for column in train_string_columns]


pipeline = Pipeline(stages=indexers)
train_indexed = pipeline.fit(df).transform(df)

In [20]:
train_indexed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Registration State: integer (nullable = true)
 |-- Plate Type: integer (nullable = true)
 |-- Issue Date: integer (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: integer (nullable = true)
 |-- Vehicle Make: integer (nullable = true)
 |-- Issuing Agency: integer (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Command: integer (nullable = true)
 |-- Violation In Front Of Or Opposite: integer (nullable = true)
 |-- Violation County: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)



In [21]:
# Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ['Registration State',
                'Plate Type',
                'Issue Date',
                'Violation Code',
                'Vehicle Body Type',
                'Vehicle Make', 
                'Issuing Agency',
                'Street Code1','Street Code2','Street Code3','Issuer Precinct','Issuer Command','Violation In Front Of Or Opposite','Violation County']
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
assembledDF = assembler.transform(df)
assembledDF.select("features").show(5, False)

+------------------------------------------------------------------------------------+
|features                                                                            |
+------------------------------------------------------------------------------------+
|[45.0,56.0,1853.0,36.0,1009.0,593.0,12.0,260.0,265.0,243.0,114.0,2644.0,1.0,5.0]    |
|[45.0,56.0,1201.0,20.0,1009.0,1957.0,12.0,5695.0,1305.0,6246.0,67.0,2632.0,1.0,2.0] |
|[45.0,18.0,452.0,36.0,1274.0,1056.0,12.0,334.0,1210.0,1161.0,17.0,2605.0,1.0,4.0]   |
|[9.0,56.0,1131.0,20.0,1.0,3766.0,12.0,4961.0,2342.0,2566.0,84.0,2631.0,3.0,2.0]     |
|[45.0,56.0,1855.0,73.0,1009.0,1056.0,12.0,1543.0,2989.0,4435.0,120.0,2633.0,1.0,6.0]|
+------------------------------------------------------------------------------------+
only showing top 5 rows



In [22]:
# Import VectorIndexer from pyspark.ml.feature package
from pyspark.ml.feature import VectorIndexer
# Create a list of all the raw features
# VectorIndexer will automatically identify the categorical columns and index them
featurecol =['Registration State',
                'Plate Type',
                'Issue Date',
                'Violation Code',
                'Vehicle Body Type',
                'Vehicle Make', 
                'Issuing Agency',
                'Street Code1','Street Code2','Street Code3','Issuer Precinct','Issuer Command','Violation In Front Of Or Opposite','Violation County']
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= featurecol, outputCol= "features")
assembledDF = assembler.transform(df)

# Create the VectorIndexer object. It only take feature column
vecindexer = VectorIndexer(inputCol= "features", outputCol= "indexed_features")
# Fit the vectorindexer object on the output of the vectorassembler data and transform
vecindexedDF = vecindexer.fit(assembledDF).transform(assembledDF)
vecindexedDF.select("features", "indexed_features").show(5, False)

+------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|features                                                                            |indexed_features                                                                    |
+------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|[45.0,56.0,1853.0,36.0,1009.0,593.0,12.0,260.0,265.0,243.0,114.0,2644.0,1.0,5.0]    |[45.0,56.0,1853.0,36.0,1009.0,593.0,11.0,260.0,265.0,243.0,114.0,2644.0,0.0,3.0]    |
|[45.0,56.0,1201.0,20.0,1009.0,1957.0,12.0,5695.0,1305.0,6246.0,67.0,2632.0,1.0,2.0] |[45.0,56.0,1201.0,20.0,1009.0,1957.0,11.0,5695.0,1305.0,6246.0,67.0,2632.0,0.0,1.0] |
|[45.0,18.0,452.0,36.0,1274.0,1056.0,12.0,334.0,1210.0,1161.0,17.0,2605.0,1.0,4.0]   |[45.0,18.0,452.0,36.0,1274.0,1056.0,11.0,334.0,1210.0,

In [23]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
densefeatureDF = assembledDF.withColumn('features_array', sparseToDense('features'))

densefeatureDF.select("features", "features_array").show(5, False)

+------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|features                                                                            |features_array                                                                      |
+------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|[45.0,56.0,1853.0,36.0,1009.0,593.0,12.0,260.0,265.0,243.0,114.0,2644.0,1.0,5.0]    |[45.0,56.0,1853.0,36.0,1009.0,593.0,12.0,260.0,265.0,243.0,114.0,2644.0,1.0,5.0]    |
|[45.0,56.0,1201.0,20.0,1009.0,1957.0,12.0,5695.0,1305.0,6246.0,67.0,2632.0,1.0,2.0] |[45.0,56.0,1201.0,20.0,1009.0,1957.0,12.0,5695.0,1305.0,6246.0,67.0,2632.0,1.0,2.0] |
|[45.0,18.0,452.0,36.0,1274.0,1056.0,12.0,334.0,1210.0,1161.0,17.0,2605.0,1.0,4.0]   |[45.0,18.0,452.0,36.0,1274.0,1056.0,12.0,334.0,1210.0,

In [24]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol= "features_array", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(densefeatureDF).transform(densefeatureDF)
stdscaledDF.select("scaledfeatures" ).show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaledfeatures                                                                                                                                                                                                                                                            |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[5.376824320574023,3.5126764661411674,3.4662763117339246,1.8457517694642522,1.9776510824233242,0.5296980160889667,14.61028702700145,0.1358087605337972,0.13452539394656537,0.12382305585540042,2

In [25]:
# We spilt the data into 70-30 set
# Training Set - 70% obesevations
# Testing Set - 30% observations
trainDF, testDF =  assembledDF.randomSplit([0.7,0.3], seed = 2020)

# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

Observations in training set =  1059713
Observations in testing set =  454157


In [26]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='Violation Location', maxIter=10, 
                      regParam=0.8, elasticNetParam=0.1)
lr_model = lr.fit(trainDF)

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

lr_predictions = lr_model.transform(testDF)
lr_predictions.select("prediction","Violation Location","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Violation Location",metricName="r2")
print("R Squared (R2) on val data = %g" % lr_evaluator.evaluate(lr_predictions))

RMSE: 23.682770
r2: 0.610629
+------------------+------------------+--------------------+
|        prediction|Violation Location|            features|
+------------------+------------------+--------------------+
| 95.57229960402864|               106|[45.0,56.0,943.0,...|
| 75.33243946543661|                69|[45.0,56.0,304.0,...|
| 34.55070653975065|                12|[40.0,56.0,905.0,...|
|29.296004656288016|                19|[45.0,18.0,84.0,3...|
| 37.02457324495879|                22|[50.0,56.0,1817.0...|
+------------------+------------------+--------------------+
only showing top 5 rows

R Squared (R2) on val data = 0.601233


In [27]:
#Training XGBOOST only on first few data, since training happens outside pyspark
from xgboost import XGBClassifier
df_pandas = df.limit(100000).toPandas()
column_names = ['Registration State',
                'Plate Type',
                'Issue Date',
                'Violation Code',
                'Vehicle Body Type',
                'Vehicle Make', 
                'Issuing Agency',
                'Street Code1','Street Code2','Street Code3','Issuer Precinct','Issuer Command','Violation In Front Of Or Opposite','Violation County']
X_pandas = df_pandas[column_names].values
y_pandas = df_pandas['Violation Location'].values
X_train, X_test, y_train, y_test = train_test_split(X_pandas, y_pandas, test_size = 0.2)
xgboost_model = XGBClassifier(max_depth=7, n_estimators=100, objective='multi:softprob')
xgboost_model.fit(X_train, y_train)

XGBClassifier(max_depth=7, objective='multi:softprob')

In [ ]:
#Testing the model
from sklearn.metrics import accuracy_score, f1_score ,balanced_accuracy_score
y_pred_train = xgboost_model.predict(X_train)
print("For train\nAccuracy score:{},Balanced accuracy score:{},f1 score:{}".format(accuracy_score(y_train,y_pred_train),balanced_accuracy_score(y_train,y_pred_train),f1_score(y_train,y_pred_train, average='weighted')))
y_pred_test = xgboost_model.predict(X_test)
print("For test\nAccuracy score:{},Balanced accuracy score:{},f1 score:{}".format(accuracy_score(y_test,y_pred_test),balanced_accuracy_score(y_test,y_pred_test),f1_score(y_test,y_pred_test, average = 'weighted')))

For train
Accuracy score:0.9998875,Balanced accuracy score:0.9396310021137639,f1 score:0.9998558517007085
For test
Accuracy score:0.98805,Balanced accuracy score:0.9486424244892658,f1 score:0.9880298968350442


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [ ]:
import pickle
pickle.dump(xgboost_model, open('model.bin', 'wb'))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
t_predictonDF = lr_model.transform(stdscaledDF)

In [ ]:
t_predictonDF.select('Prediction').show(5)

+------------------+
|        Prediction|
+------------------+
| 86.83670867823407|
| 68.98968422794032|
| 21.04581701766899|
| 71.08622264236064|
|102.49274196822653|
+------------------+
only showing top 5 rows



In [ ]:
df.columns

['_c0',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Issuing Agency',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Issuer Precinct',
 'Issuer Command',
 'Violation In Front Of Or Opposite',
 'Violation County',
 'Violation Location']

In [ ]:
p_test = df.toPandas()
df_res = p_test['Violation Location']

In [ ]:
df.withColumnRenamed("Violation Location","Violation_Location").printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Registration State: integer (nullable = true)
 |-- Plate Type: integer (nullable = true)
 |-- Issue Date: integer (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: integer (nullable = true)
 |-- Vehicle Make: integer (nullable = true)
 |-- Issuing Agency: integer (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Command: integer (nullable = true)
 |-- Violation In Front Of Or Opposite: integer (nullable = true)
 |-- Violation County: integer (nullable = true)
 |-- Violation_Location: integer (nullable = true)



In [ ]:
#res = t_predictonDF.select('Prediction')
#import pandas as pd
#res = pd.concat([df_res,t_predictonDF.select('Prediction')],axis=1)

In [ ]:
t_predictonDF.select('Prediction').show(5)

+------------------+
|        Prediction|
+------------------+
| 86.83670867823407|
| 68.98968422794032|
| 21.04581701766899|
| 71.08622264236064|
|102.49274196822653|
+------------------+
only showing top 5 rows

